In [202]:
import numpy as np
from osgeo import gdal
from osgeo import osr

#-------------------------------
#file einlesen:
#-------------------------------
fobj = open('Daten/Waldpunktwolke_100.txt', "r")

pointlist = []

next(fobj)
for line in fobj:
    line = line.strip()
    line= line.split("\t")   
    pointlist.append(line)
fobj.close

pointarray = np.array(pointlist) 
pointarray = pointarray.astype(float)  


#Neuer Array ohne Gebauede und Boden, Klassen 3, 4, 5
vegetationslist = []

for i in pointarray:
        
        if i[6] == 3 or i[6] ==4 or i[6] ==5:
                vegetationslist.append(i)
                

vegetationsarray = np.array(vegetationslist)        


#Neuer Array Bodenpunkte
bodenlist = []

for i in pointarray:
        
        if i[6] == 2:
                bodenlist.append(i)
                
bodenarray = np.array(bodenlist)


#----------------------------------
#Extend Array with random numbers:
#----------------------------------
#max values
maxvals = np.amax(pointarray,axis=0) 
xmax=maxvals[0]
ymax=maxvals[1]
# print ("xmax:",xmax)
# print ("ymax:",ymax)

#min values
minvals = np.amin(pointarray,axis=0) 
xmin=minvals[0]
ymin=minvals[1]
# print ("xmin:",xmin)
# print ("ymin:",ymin)

#empty array with extent dimensions
nrows= int(np.ceil(ymax-ymin))
ncols= int(np.ceil(xmax-xmin))

bodenout_array = np.empty((nrows,ncols))
bodenout_array.fill(0.0)

#----------------------------------
#Bodenpunkte zaehlen:
#----------------------------------
bodencount = 0
countlist =[]
arraycount_x = 0
arraycount_y = 0

for y in np.arange(ymax,ymax-nrows,-1):
    # print("yRichtung:",y)
    # print("ArraypositionY:",arraycount_y)
    for x in np.arange(xmin,xmin+ncols):
        # print("xRichtung:",x)
        # print("ArraypositionX:",arraycount_x)
        for i in bodenarray:
            if i[0] >= x and i[0] < x+1 and i[1] <= y and i[1] > y-1:
                        bodencount+=1
        bodenout_array[arraycount_y][arraycount_x]=bodencount
        arraycount_x+=1     
        bodencount=0
    arraycount_x=0
    arraycount_y+=1
    # print("xRichtungEnde")    




In [203]:
#-------------------------------
#OutRaster schreiben:
#-------------------------------

driver = gdal.GetDriverByName("GTiff")
dataset = driver.Create("Out.tif", int(ncols), int(nrows), 1, gdal.GDT_Float32)
dataset.SetGeoTransform((xmin,1,0,ymax,0,-1))

#Koordinatensystem definieren:
dstSRS = osr.SpatialReference()
dstSRS.ImportFromEPSG(32623)
dest_wkt = dstSRS.ExportToWkt()

dataset.SetProjection(dest_wkt)

#Raster ausgeben:
bandout = dataset.GetRasterBand(1).WriteArray(bodenout_array)
dataset.FlushCache()